In [65]:
import numpy as np
import pandas as pd
import tpqoa
from datetime import datetime, timedelta
import time

In [86]:
class ConTrader(tpqoa.tpqoa):
    
    def __init__(self, config_file, instrument, bar_length, window, units):
        super().__init__(config_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #################### Strategy Specific Attributes ####################
        self.window = window
        ######################################################################
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)

            df = self.get_history(
                instrument = self.instrument,
                start = past,
                end = now,
                granularity = "S5",
                price = "M",
                localize = False
            ).c.dropna().to_frame()

            df.rename(columns = {"c" : self.instrument}, inplace = True)

            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[ : -1]

            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument : (ask + bid) / 2},
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, label = "right").last().ffill())
        if len(self.raw_data) > 1:
            self.raw_data = self.raw_data.iloc[ : -1]
        self.tick_data = self.tick_data.iloc[-1 :]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self):
        df = self.raw_data.copy()
        
        #################### Contrarian Strategy ####################
        df["Returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["Position"] = -np.sign(df.Returns.rolling(self.window).mean())
        #############################################################
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["Position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, 2 * self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["Position"].iloc[-1] == -1:
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -2 * self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["Position"].iloc[-1] == 0:
            if self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, message):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100 * "-")
        print(f"{time} | {message}")
        print(f"{time} | Units = {units} | Price = {price} | P&L = {pl} | Cumulative P&L = {cumpl}")
        print(100 * "-" + "\n")
                

In [87]:
agent = ConTrader("oanda.cfg", "EUR_USD", "1m", window = 1, units = 100000)

In [88]:
agent.get_most_recent()
agent.stream_data(agent.instrument, stop = 50)
if agent.position != 0:
    close_order = agent.create_order(agent.instrument, -agent.position * agent.units, suppress = True, ret = True)
    agent.report_trade(close_order, "GOING NEUTRAL")
    agent.position = 0

1 2 
----------------------------------------------------------------------------------------------------
2022-02-03T03:39:19.731374780Z | GOING LONG
2022-02-03T03:39:19.731374780Z | Units = 100000.0 | Price = 1.13065 | P&L = 0.0 | Cumulative P&L = 0.0
----------------------------------------------------------------------------------------------------

3 4 5 6 7 8 9 10 11 
----------------------------------------------------------------------------------------------------
2022-02-03T03:42:48.817566553Z | GOING NEUTRAL
2022-02-03T03:42:48.817566553Z | Units = -100000.0 | Price = 1.13052 | P&L = -16.5586 | Cumulative P&L = -16.5586
----------------------------------------------------------------------------------------------------

12 13 14 15 16 17 18 19 
----------------------------------------------------------------------------------------------------
2022-02-03T03:43:00.996924749Z | GOING LONG
2022-02-03T03:43:00.996924749Z | Units = 100000.0 | Price = 1.13057 | P&L = 0.0 | Cumulati

In [83]:
agent.get_positions()

[]

In [63]:
agent.data.tail(10)

,EUR_USD
2022-02-03 02:26:00+00:00,1.130020
2022-02-03 02:27:00+00:00,1.129780
2022-02-03 02:28:00+00:00,1.129840
2022-02-03 02:29:00+00:00,1.129820
2022-02-03 02:30:00+00:00,1.129800
2022-02-03 02:31:00+00:00,1.129860
2022-02-03 02:32:00+00:00,1.129940
2022-02-03 02:33:00+00:00,1.129950
2022-02-03 02:34:00+00:00,1.129920
2022-02-03 02:36:00+00:00,1.130000


In [78]:
pd.to_datetime(datetime.utcnow()).tz_localize("UTC")

Timestamp('2022-02-03 03:19:27.848419+0000', tz='UTC')